In [190]:
import numpy as np
import pylab as plt
from pinacles.wrf_physics import kessler_split
from skopt.sampler import Lhs
from skopt.space import Space

In [191]:
dt = 2.0
xlv = 2.5E6
cp = 1004.0
ep2 = 287./461.6
svp1 = 0.6112
svp2 = 17.67
svp3 = 29.65
svpT0 = 273.15
rhow = 1000.0

ids = 1; jds = 1; kds = 1
iide = 1; jde = 1; kde = 1
ims=1; jms = 1; kms = 1
ime=8; jme=8; kme=8
its=1; jts=1; kts=1
ite=ime; jte=jme; kte=kme


T_wrf = np.zeros((ime, jme, kme), order='F', dtype=np.double)
qv_wrf = np.zeros_like(T_wrf)
qc_wrf = np.zeros_like(T_wrf)
qr_wrf = np.zeros_like(T_wrf)
rho_wrf = np.zeros_like(T_wrf)
exner_wrf = np.zeros_like(T_wrf)




z = np.zeros_like(T_wrf)
dz_wrf = np.zeros_like(T_wrf)

RAINNC = np.zeros((ime, jme), order='F', dtype=np.double)
RAINNCV = np.zeros_like(RAINNC)

In [192]:
#lhs = Lhs(lhs_type="classic", criterion=None)
#space = Space([(273.15, 310.),   
#               (1e-6, 2e-2),
#               (-1e-6, 1e-2),
#               (-1e-6, 1e-3),
#               (0.3, 1e5), 
#               (0.3, 1.0)])

#n_samples = ime * jme * kme
#x = lhs.generate(space.dimensions, n_samples)

#count = 0
#for i in range(ime): 
#    for j in range(jme):
#        for k in range(kme):
#            T_wrf[i,j,k] = x[count][0]
#            qv_wrf[i,j,k] = x[count][1]
#            qc_wrf[i,j,k] = x[count][2]
#            qr_wrf[i,j,k] = x[count][3]
#            rho_wrf[i,j,k] = x[count][4]
#            exner_wrf[i,j,k] = x[count][5]
#            count += 1

#T_input = T_wrf.flatten('F')
#qv_input = qv_wrf.flatten('F')
#qc_input = qc_wrf.flatten('F')
#qr_input = qr_wrf.flatten('F')
#rho_input = rho_wrf.flatten('F')
#exner_input = exner_wrf.flatten('F')

#print(T_input.shape)
           

In [193]:


#First generat data for non_zero all condensate
lhs = Lhs(lhs_type="classic", criterion=None)
space = Space([(273.15, 310.),   
               (np.log(1e-6), np.log(2.1e-2)),
               (0.3, 1.3), 
               (0.8, 1.0)])

n_samples = ime * jme * kme
x = lhs.generate(space.dimensions, n_samples)

count = 0
for i in range(ime): 
    for j in range(jme):
        for k in range(kme):
            T_wrf[i,j,k] = x[count][0]/x[count][3]
            qv_wrf[i,j,k] = np.exp(x[count][1])
            qc_wrf[i,j,k] = 0.0
            qr_wrf[i,j,k] = 0.0
            rho_wrf[i,j,k] = x[count][2]
            exner_wrf[i,j,k] =  x[count][3]
            count += 1

T_input = T_wrf.flatten('F')
qv_input = qv_wrf.flatten('F')
qc_input = qc_wrf.flatten('F')
qr_input = qr_wrf.flatten('F')
rho_input = rho_wrf.flatten('F')
exner_input = exner_wrf.flatten('F')



kessler_split.module_mp_kessler.kessler_source(T_wrf, qv_wrf, qc_wrf, qr_wrf, rho_wrf, exner_wrf,
            dt, z, xlv, cp,
            ep2, svp1, svp2, svp3, svpT0, rhow,
            dz_wrf,
            RAINNC, RAINNCV,
            ids,iide, jds,jde, kds,kde,
            ims,ime, jms,jme, kms,kme,
            its,ite, jts,jte, kts,kte)


T_output = T_wrf.flatten('F')
qv_output = qv_wrf.flatten('F')
qc_output = qc_wrf.flatten('F')
qr_output = qr_wrf.flatten('F')
rho_output = rho_wrf.flatten('F')
exner_output = exner_wrf.flatten('F')

#First generat data for non_zero all condensate
lhs = Lhs(lhs_type="classic", criterion=None)
space = Space([(273.15, 310.),   
               (np.log(1e-6), np.log(2.1e-2)),
               (np.log(1e-13), np.log(1e-2)),
               (np.log(1e-13), np.log(1e-4)),
               (0.3, 1.3), 
               (0.8, 1.0)])

n_samples = ime * jme * kme
x = lhs.generate(space.dimensions, n_samples)


count = 0
for i in range(ime): 
    for j in range(jme):
        for k in range(kme):
            T_wrf[i,j,k] = x[count][0]/x[count][5]
            qv_wrf[i,j,k] = np.exp(x[count][1])
            qc_wrf[i,j,k] =  np.exp(x[count][2])
            qr_wrf[i,j,k] =  np.exp(x[count][3])
            rho_wrf[i,j,k] =  x[count][4]
            exner_wrf[i,j,k] =  x[count][5]
            count += 1

T_input = np.append(T_input, T_wrf.flatten('F'))
qv_input = np.append(qv_input , qv_wrf.flatten('F'))
qc_input = np.append(qc_input, qc_wrf.flatten('F'))
qr_input = np.append(qr_input, qr_wrf.flatten('F'))
rho_input = np.append(rho_input, rho_wrf.flatten('F'))
exner_input = np.append(exner_input, exner_wrf.flatten('F'))


print(np.shape(T_input))

kessler_split.module_mp_kessler.kessler_source(T_wrf, qv_wrf, qc_wrf, qr_wrf, rho_wrf, exner_wrf,
            dt, z, xlv, cp,
            ep2, svp1, svp2, svp3, svpT0, rhow,
            dz_wrf,
            RAINNC, RAINNCV,
            ids,iide, jds,jde, kds,kde,
            ims,ime, jms,jme, kms,kme,
            its,ite, jts,jte, kts,kte)

T_output = np.append(T_output, T_wrf.flatten('F'))
qv_output = np.append(qv_output, qv_wrf.flatten('F'))
qc_output = np.append(qc_output, qc_wrf.flatten('F'))
qr_output = np.append(qr_output, qr_wrf.flatten('F'))
rho_output = np.append(rho_output, rho_wrf.flatten('F'))
exner_output = np.append(exner_output, exner_wrf.flatten('F'))




#Now let's generate for when there is no rain
space = Space([(273.15, 310.),   
               (np.log(1e-6), np.log(2.1e-2)),
               (np.log(1e-13), np.log(1e-2)),
               (0.3, 1.3), 
               (0.8, 1.0)])

n_samples = ime * jme * kme
x = lhs.generate(space.dimensions, n_samples)


count = 0
for i in range(ime): 
    for j in range(jme):
        for k in range(kme):
            T_wrf[i,j,k] = x[count][0]/x[count][4]
            qv_wrf[i,j,k] = np.exp(x[count][1])
            qc_wrf[i,j,k] =  np.exp(x[count][2])
            qr_wrf[i,j,k] =  0.0 #np.exp(x[count][3])
            rho_wrf[i,j,k] =  x[count][3]
            exner_wrf[i,j,k] =  x[count][4]
            count += 1

T_input = np.append(T_input, T_wrf.flatten('F'))
qv_input = np.append(qv_input , qv_wrf.flatten('F'))
qc_input = np.append(qc_input, qc_wrf.flatten('F'))
qr_input = np.append(qr_input, qr_wrf.flatten('F'))
rho_input = np.append(rho_input, rho_wrf.flatten('F'))
exner_input = np.append(exner_input, exner_wrf.flatten('F'))


print(np.shape(T_input))

kessler_split.module_mp_kessler.kessler_source(T_wrf, qv_wrf, qc_wrf, qr_wrf, rho_wrf, exner_wrf,
            dt, z, xlv, cp,
            ep2, svp1, svp2, svp3, svpT0, rhow,
            dz_wrf,
            RAINNC, RAINNCV,
            ids,iide, jds,jde, kds,kde,
            ims,ime, jms,jme, kms,kme,
            its,ite, jts,jte, kts,kte)

T_output = np.append(T_output, T_wrf.flatten('F'))
qv_output = np.append(qv_output, qv_wrf.flatten('F'))
qc_output = np.append(qc_output, qc_wrf.flatten('F'))
qr_output = np.append(qr_output, qr_wrf.flatten('F'))
rho_output = np.append(rho_output, rho_wrf.flatten('F'))
exner_output = np.append(exner_output, exner_wrf.flatten('F'))


#Now let's generate for when there is no cloud
space = Space([(273.15, 310.),   
               (np.log(1e-6), np.log(2.1e-2)),
               (np.log(1e-13), np.log(1e-4)),
               (0.3, 1.3), 
               (0.8, 1.0)])

n_samples = ime * jme * kme
x = lhs.generate(space.dimensions, n_samples)


count = 0
for i in range(ime): 
    for j in range(jme):
        for k in range(kme):
            T_wrf[i,j,k] = x[count][0]/x[count][4]
            qv_wrf[i,j,k] = np.exp(x[count][1])
            qc_wrf[i,j,k] =  0.0
            qr_wrf[i,j,k] =  np.exp(x[count][2])
            rho_wrf[i,j,k] =  x[count][3]
            exner_wrf[i,j,k] =  x[count][4]
            count += 1

T_input = np.append(T_input, T_wrf.flatten('F'))
qv_input = np.append(qv_input , qv_wrf.flatten('F'))
qc_input = np.append(qc_input, qc_wrf.flatten('F'))
qr_input = np.append(qr_input, qr_wrf.flatten('F'))
rho_input = np.append(rho_input, rho_wrf.flatten('F'))
exner_input = np.append(exner_input, exner_wrf.flatten('F'))


print(np.shape(T_input))

kessler_split.module_mp_kessler.kessler_source(T_wrf, qv_wrf, qc_wrf, qr_wrf, rho_wrf, exner_wrf,
            dt, z, xlv, cp,
            ep2, svp1, svp2, svp3, svpT0, rhow,
            dz_wrf,
            RAINNC, RAINNCV,
            ids,iide, jds,jde, kds,kde,
            ims,ime, jms,jme, kms,kme,
            its,ite, jts,jte, kts,kte)

T_output = np.append(T_output, T_wrf.flatten('F'))
qv_output = np.append(qv_output, qv_wrf.flatten('F'))
qc_output = np.append(qc_output, qc_wrf.flatten('F'))
qr_output = np.append(qr_output, qr_wrf.flatten('F'))
rho_output = np.append(rho_output, rho_wrf.flatten('F'))
exner_output = np.append(exner_output, exner_wrf.flatten('F'))


(1024,)
(1536,)
(2048,)


In [194]:
#plt.scatter(qc_input, T_input, c=T_output-T_input)
#plt.colorbar()

In [195]:
training_data = {}
training_data['T_input'] = T_input
training_data['qv_input'] = qv_input
training_data['qc_input'] = qc_input
training_data['qr_input'] = qr_input
training_data['rho_input'] = rho_input
training_data['exner_input'] = exner_input

training_data['T_output'] = T_output
training_data['qv_output'] = qv_output
training_data['qc_output'] = qc_output
training_data['qr_output'] = qr_output
training_data['rho_output'] = rho_output 
training_data['exner_output'] = exner_output



import pickle as pkl
with open('kessler_trianing.pkl', 'wb') as f:
    pkl.dump(training_data, f)



110592